### Importa a base de dados

In [101]:
url = r"C:\Users\joth1\Downloads\Nova pasta (2)\base_dados_sptrans_passageiros.csv"

import pandas as pd

df = pd.read_csv(url, sep=';')
df['Partição'] = df['Partição'].fillna('0').astype(int)

#  Trato os dados
df['Total'] = df['Total'].str.replace('.', '')
df['Subsistema Estrutural'] = df['Subsistema Estrutural'].str.replace('.', '')
df['Subsistema Local'] = df['Subsistema Local'].str.replace('.', '')

df['Total'] = df['Total'].astype(int)
df['Subsistema Estrutural'] = df['Subsistema Estrutural'].astype(int)
df['Subsistema Local'] = df['Subsistema Local'].astype(int)

#  Agrupo, trazendo o total e a representatividade dos sistemas Locais, dentro de São Paulo
df_2 = df.groupby(['Partição']).agg(
    qtd=('Total', 'sum')
    ,qtd_2=('Subsistema Estrutural', 'sum')
    ,qtd_3=('Subsistema Local', 'sum')
)

df_2['diferenca'] = df_2['qtd_3'] / df_2['qtd']  * 100
df_2

,qtd,qtd_2,qtd_3,diferenca
Partição,,,,
2006,2661110194,1569497687,1091612507,41.020943
2007,2731968253,1590463119,1141505134,41.783250
2008,2835856140,1734267048,1101589092,38.845027
2009,2870007561,1715876626,1154130935,40.213515
2010,2915990761,1712978513,1203012248,41.255695
2011,2940894817,1710098842,1230795975,41.851071
2012,2916954960,1647064166,1269890794,43.534810


In [104]:
import numpy as np
# Criando uma tabela dinâmica
pivot_table = df_2.pivot_table(values='diferenca', index='Partição', columns='Partição', aggfunc=np.sum)

# Substituindo os valores NaN por 0
pivot_table = pivot_table.fillna(0)


# Aplicando a coloração com a escala de verde para vermelho
styled_table = pivot_table.style.background_gradient(cmap='RdYlGn_r')  # 'RdYlGn_r' é a escala do vermelho ao verde

# Exibindo a tabela estilizada
styled_table

C:\Users\joth1\AppData\Local\Temp\ipykernel_25716\3103880233.py:3: FutureWarning: The provided callable <function sum at 0x00000243172C11C0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = df_2.pivot_table(values='diferenca', index='Partição', columns='Partição', aggfunc=np.sum)


Partição,2006,2007,2008,2009,2010,2011,2012
Partição,,,,,,,
2006,41.020943,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007,0.000000,41.783250,0.000000,0.000000,0.000000,0.000000,0.000000
2008,0.000000,0.000000,38.845027,0.000000,0.000000,0.000000,0.000000
2009,0.000000,0.000000,0.000000,40.213515,0.000000,0.000000,0.000000
2010,0.000000,0.000000,0.000000,0.000000,41.255695,0.000000,0.000000
2011,0.000000,0.000000,0.000000,0.000000,0.000000,41.851071,0.000000
2012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43.534810


In [2]:
import pandas as pd
url2 = r"C:\Users\joth1\Downloads\Nova pasta (2)\routes.txt"
df_linhas = pd.read_csv(url2)

In [3]:
df_linhas

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF
1,1012-21,1,1012-21,Term. Jd. Britânia - Jd. Rosinha,3,509E2F,FFFFFF
2,1014-10,1,1014-10,Morro Doce - Cptm Perus,3,509E2F,FFFFFF
3,1015-10,1,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,3,509E2F,FFFFFF
4,1016-10,1,1016-10,Cem. Do Horto - Shop. Center Norte,3,002F6C,FFFFFF
...,...,...,...,...,...,...,...
1334,N839-11,1,N839-11,Metrô Butantã - Metrô Vl. Mariana,3,FF671F,000000
1335,N840-11,1,N840-11,Term. Vl. Mariana - Sta. Cecília,3,FF671F,000000
1336,N841-11,1,N841-11,Term. Vl. Mariana - Sta. Cecília,3,FF671F,000000
1337,N842-11,1,N842-11,Term. Pinheiros - Cohab Raposo Tavares,3,FF671F,000000


In [17]:
df = df_linhas[['route_id', 'route_long_name']]
df

,route_id,route_long_name
0,1012-10,Term. Jd. Britania - Jd. Monte Belo
1,1012-21,Term. Jd. Britânia - Jd. Rosinha
2,1014-10,Morro Doce - Cptm Perus
3,1015-10,Term. Jd. Britânia - Chác. Maria Trindade
4,1016-10,Cem. Do Horto - Shop. Center Norte
...,...,...
1334,N839-11,Metrô Butantã - Metrô Vl. Mariana
1335,N840-11,Term. Vl. Mariana - Sta. Cecília
1336,N841-11,Term. Vl. Mariana - Sta. Cecília
1337,N842-11,Term. Pinheiros - Cohab Raposo Tavares


In [18]:
### Encontra a latitude e longitude 

In [24]:
import requests

def get_coordinates_by_bairro_nominatim(bairro, cidade, estado):
    endereco = f"{bairro}, {cidade}, {estado}"
    url = f'https://nominatim.openstreetmap.org/search?format=json&q={endereco}'
    
    response = requests.get(url)
    data = response.json()
    
    if data:
        # Extrai a latitude e longitude do primeiro resultado
        latitude = float(data[0]['lat'])
        longitude = float(data[0]['lon'])
        return latitude, longitude
    return None, None

# Exemplo de uso
bairro = 'Vila Mariana'
cidade = 'São Paulo'
estado = 'SP'
latitude, longitude = get_coordinates_by_bairro_nominatim(bairro, cidade, estado)
print(f'A latitude é: {latitude}, A longitude é: {longitude}')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Achar o Bairro pela latitude e Longitude 

In [41]:
import requests

def get_bairro_by_coordinates(lat, lon):
    url = f'https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json&addressdetails=1'
    
    # Adicionando o cabeçalho 'User-Agent' para evitar o erro 403
    headers = {
        'User-Agent': 'SeuNome/SeuEmail'  # Substitua com seu nome e e-mail ou nome do projeto
    }
    
    response = requests.get(url, headers=headers)
    
    # Verifica se a resposta foi bem-sucedida (status code 200)
    if response.status_code == 200:
        try:
            data = response.json()  # Tenta converter a resposta para JSON
            if 'address' in data:
                # Tentamos diferentes campos que podem conter o bairro ou a região
                bairro = data['address'].get('neighbourhood', None)
                if not bairro:  # Se não encontrar bairro, tenta 'suburb', 'city_district', ou 'municipality'
                    bairro = data['address'].get('suburb', None)
                    if not bairro:
                        bairro = data['address'].get('city_district', None)
                        if not bairro:
                            bairro = data['address'].get('municipality', None)

                if bairro:
                    return bairro
                else:
                    return "Bairro não encontrado"
            return "Informações não disponíveis"
        except ValueError:
            return "Erro ao decodificar a resposta JSON."
    elif response.status_code == 403:
        return "Erro 403: Acesso negado. Verifique as limitações da API."
    else:
        return f"Erro na requisição: {response.status_code}"

# Exemplo de uso
latitude = -23.5824973  # Latitude para a Avenida Paulista
longitude = -46.4092065 # Longitude para a Avenida Paulista

bairro = get_bairro_by_coordinates(latitude, longitude)
print(f'O bairro é: {bairro}')

O bairro é: Cidade Tiradentes


### Achar a latitude e Longitude pelo nome

In [40]:
import requests

def get_coordinates_by_bairro(bairro):
    # URL para fazer a requisição à API Nominatim com o nome do bairro
    url = f'https://nominatim.openstreetmap.org/search?q={bairro}&format=json&addressdetails=1'
    
    # Cabeçalho 'User-Agent' necessário para evitar erro 403
    headers = {
        'User-Agent': 'SeuNome/SeuEmail'  # Substitua pelo seu nome e email ou o nome do seu projeto
    }
    
    # Fazendo a requisição
    response = requests.get(url, headers=headers)
    
    # Verifica se a resposta foi bem-sucedida (status code 200)
    if response.status_code == 200:
        try:
            data = response.json()  # Converte a resposta para JSON
            if data:
                # A primeira entrada da lista geralmente é a mais precisa
                result = data[0]  # Pega o primeiro resultado encontrado
                lat = result['lat']  # Latitude
                lon = result['lon']  # Longitude
                
                return lat, lon
            else:
                return "Bairro não encontrado"
        except ValueError:
            return "Erro ao decodificar a resposta JSON."
    elif response.status_code == 403:
        return "Erro 403: Acesso negado. Verifique as limitações da API."
    else:
        return f"Erro na requisição: {response.status_code}"

# Exemplo de uso
bairro = "Cidade Tiradentes, São Paulo"  # Nome do bairro que você deseja buscar

# Chama a função e imprime a latitude e longitude
coordenadas = get_coordinates_by_bairro(bairro)
if isinstance(coordenadas, tuple):
    print(f'Latitude: {coordenadas[0]}, Longitude: {coordenadas[1]}')
else:
    print(coordenadas)


Latitude: -23.5824973, Longitude: -46.4092065
